In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import statsapi
import pandas as pd
import numpy as np
from pathlib import Path
from tqdm import tqdm

In [3]:
start_date = '02/01/2023'  # mm/dd/yyyy
end_date = '10/01/2023'  # mm/dd/yyyy
data_dir = Path('data')  # path/to/save/location

In [4]:
stats_games = pd.DataFrame(statsapi.schedule(date=None, start_date=start_date, end_date=end_date, team="", opponent="", sportId=1, game_id=None))
pitchers = pd.concat([stats_games.winning_pitcher, stats_games.losing_pitcher], ignore_index=True).unique()

In [5]:
stats_pitchers = []
for pitcher_name in tqdm(pitchers):
    try:
        pitcher_id = statsapi.lookup_player(pitcher_name)[0]['id']
        stats = statsapi.player_stat_data(pitcher_id, group="[pitching]", type="season", sportId=1)
        
        stats_pitchers.append({
            'pitcher_id': pitcher_id,
            'pitcher_name': pitcher_name,
            'current_team': stats['current_team'],
            'position': stats['position'],
            'pitch_hand': stats['pitch_hand'],
            'games_started': stats['stats'][0]['stats']['gamesStarted']        
        })
    except:
        pass
    
    

100%|██████████| 799/799 [11:53<00:00,  1.12it/s]


In [7]:
stats_pitchers = pd.DataFrame(stats_pitchers)

In [9]:
stats_pitchers

,pitcher_id,pitcher_name,current_team,position,pitch_hand,games_started
0,661395,Jhoan Duran,Minnesota Twins,P,Right,0
1,472551,NaN,Colorado Rockies,P,Left,0
2,605130,Scott Barlow,Kansas City Royals,P,Right,0
3,656353,Tucker Davidson,Los Angeles Angels,P,Left,0
4,665625,Elvis Peguero,Milwaukee Brewers,P,Right,0
...,...,...,...,...,...,...
355,663362,Matt Waldron,El Paso Chihuahuas,P,Right,1
356,676265,Cory Abbott,Washington Nationals,P,Right,0
357,519008,T.J. McFarland,New York Mets,P,Left,0
358,607074,Carlos Rodon,New York Yankees,P,Left,1


In [16]:
stats_games.to_csv(data_dir / 'stats_games.csv')
stats_pitchers.to_csv(data_dir / 'stats_pitchers.csv')